In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, ndcg_at_k
import warnings

In [ ]:
class UltraVibeRecommender:
    def __init__(self):
        # Инициализация хранилища параметров модели
        self.model = None            # Объект обученной модели ALS
        self.user_item_matrix = None # CSR-матрица пользователь-трек (логарифмированные взаимодействия)
        self.user_to_idx = {}        # Маппинг user_id -> индекс строки в матрице
        self.track_to_idx = {}       # Маппинг track_id -> индекс столбца в матрице
        self.idx_to_track = {}       # Обратный маппинг индекса -> track_id
        self.tracks_df = None        # Метаданные треков (name, artists)
        self.is_fitted = False      # Флаг завершения обучения

    def load_and_prepare(self, interactions_path, tracks_path, catalog_path, top_users=5000, top_tracks=3000):
        """Загрузка и предобработка данных.
        
        Args:
            top_users: Порог отбора пользователей по количеству взаимодействий
            top_tracks: Порог отбора треков по популярности
            interactions_path: Путь к данным о взаимодействиях user-track
            tracks_path: Путь к техническим метаданным треков
            catalog_path: Путь к справочнику названий
        """
        # Загрузка исходных данных из Parquet
        interactions = pd.read_parquet(interactions_path)
        tracks = pd.read_parquet(tracks_path)
        catalog = pd.read_parquet(catalog_path)

        # Фильтрация неактивных пользователей и непопулярных треков
        user_counts = interactions['user_id'].value_counts().head(top_users)
        track_counts = interactions['track_id'].value_counts().head(top_tracks)
        filtered = interactions[
            (interactions['user_id'].isin(user_counts.index)) & 
            (interactions['track_id'].isin(track_counts.index))
        ]
        
        # Формирование матрицы признаков треков
        track_info = tracks.merge(
            catalog[catalog['type'] == 'track'],
            left_on='track_id', right_on='id', how='left'
        )
        self.tracks_df = track_info.set_index('track_id')

        # Создание биективных отображений ID -> индекс
        unique_users = filtered['user_id'].unique()
        unique_tracks = filtered['track_id'].unique()
        self.user_to_idx = {u: i for i, u in enumerate(unique_users)}
        self.track_to_idx = {t: i for i, t in enumerate(unique_tracks)}
        self.idx_to_track = {i: t for t, i in self.track_to_idx.items()}

        # Построение разреженной матрицы взаимодействий
        grp = filtered.groupby(['user_id', 'track_id']).size().reset_index(name='count')
        grp['count'] = np.log1p(grp['count'])  # Логарифмирование для нормализации
        rows = grp['user_id'].map(self.user_to_idx)
        cols = grp['track_id'].map(self.track_to_idx)
        self.user_item_matrix = csr_matrix(
            (grp['count'], (rows, cols)),
            shape=(len(unique_users), len(unique_tracks))
        )

    def fit(self, factors=128, regularization=0.01, iterations=40, alpha=20):
        """Обучение модели Alternating Least Squares.
        
        Args:
            factors: Размерность латентного пространства
            regularization: Коэффициент L2-регуляризации
            iterations: Максимальное число итераций оптимизации
            alpha: Параметр масштабирования для неявного фидбека
        """
        self.model = AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations=iterations,
            alpha=alpha,
            random_state=42
        )
        self.model.fit(self.user_item_matrix.T)  # Транспонирование для item-user формата
        self.is_fitted = True

    def recommend_by_names(self, track_names, n=10):
        """Генерация рекомендаций по списку названий треков.
        
        Args:
            track_names: Список названий для поиска в каталоге
            n: Количество возвращаемых рекомендаций
            
        Returns:
            Список словарей с полями name, artists, score
        """
        # Поиск track_id по подстроке в названиях
        found_tracks = []
        for name in track_names:
            matches = self.tracks_df[self.tracks_df['name'].str.contains(name, case=False, na=False)]
            for track_id in matches.index:
                if track_id in self.track_to_idx:
                    found_tracks.append(track_id)
                    break
        
        # Формирование вектора предпочтений пользователя
        user_vec = np.zeros(len(self.track_to_idx))
        for tid in found_tracks:
            user_vec[self.track_to_idx[tid]] = 1.0
            
        # Получение рекомендаций с фильтрацией исходных треков
        item_indices, scores = self.model.recommend(
            userid=0,
            user_items=csr_matrix(user_vec),
            N=n + len(found_tracks),
            filter_already_liked_items=True
        )
        
        # Постобработка результатов
        results = []
        for idx, score in zip(item_indices, scores):
            track_id = self.idx_to_track.get(idx)
            if track_id and track_id not in found_tracks and len(results) < n:
                info = self.tracks_df.loc[track_id]
                results.append({
                    'name': info.get('name', 'Unknown'),
                    'artists': info.get('artists', 'Unknown'),
                    'score': float(score)
                })
        return results

    def evaluate(self, k=10):
        """Оценка качества модели на отложенной выборке.
        
        Args:
            k: Количество рекомендаций для расчета метрик
            
        Returns:
            Кортеж (precision@k, ndcg@k, recall@k)
        """
        # Стратифицированное разбиение данных
        train_mat, test_mat = leave_k_out_split(self.user_item_matrix.T, K=1)
        
        # Переобучение на усеченном датасете
        temp_model = AlternatingLeastSquares(
            factors=self.model.factors,
            regularization=self.model.regularization,
            iterations=self.model.iterations,
            alpha=self.model.alpha,
            random_state=42
        )
        temp_model.fit(train_mat)
        
        # Расчет метрик качества
        prec = precision_at_k(temp_model, train_mat, test_mat, K=k)
        ndcg_score = ndcg_at_k(temp_model, train_mat, test_mat, K=k)
        recall = self.manual_recall_at_k(temp_model, train_mat, test_mat, K=k)
        return prec, ndcg_score, recall

    def manual_recall_at_k(self, model, train_mat, test_mat, K=10):
        """Кастомный расчет Recall@K для оценки релевантности рекомендаций."""
        hits = 0
        total_test_items = 0
        for user in range(test_mat.shape[1]):
            test_items = test_mat[:, user].nonzero()[0]
            if not test_items:
                continue
                
            # Генерация рекомендаций
            user_profile = train_mat[:, user].T.tocsr()
            recommended, _ = model.recommend(
                userid=0,
                user_items=user_profile,
                N=K,
                filter_already_liked_items=True
            )
            
            # Подсчет пересечений с тестовым набором
            hits += np.intersect1d(recommended, test_items).size
            total_test_items += len(test_items)
            
        return hits / total_test_items if total_test_items > 0 else 0.0

In [ ]:
def main():
    # Пути к файлам данных: взаимодействия пользователей, метаданные треков, справочник названий
    interactions_path = "/Users/emiliaaleksanyan/Desktop/coursework/interactions.parquet"
    tracks_path = "/Users/emiliaaleksanyan/Desktop/coursework/tracks (1).parquet"
    catalog_path = "/Users/emiliaaleksanyan/Desktop/coursework/catalog_names.parquet"

    # Инициализация рекомендательной системы
    recommender = UltraVibeRecommender()
    
    # Загрузка данных с фильтрацией топ-5000 пользователей и топ-3000 треков 
    # для уменьшения вычислительной нагрузки
    recommender.load_and_prepare(
        interactions_path, 
        tracks_path, 
        catalog_path,
        top_users=5000,    # Порог отбора активных пользователей
        top_tracks=3000    # Порог отбора популярных треков
    )
    
    # Обучение модели ALS с параметрами:
    recommender.fit(
        factors=128,         # Размерность латентных факторов
        regularization=0.01, # Коэффициент L2-регуляризации
        iterations=40,      # Количество итераций оптимизации
        alpha=20            # Параметр доверия для неявного фидбека
    )
    
    # Оценка качества модели по метрикам precision@10 и recall@10
    recommender.evaluate(k=10)
    
    print("\nГотово! Теперь можно получать рекомендации.")
    while True:
        # Интерактивный интерфейс для получения рекомендаций
        inp = input("\nВведите названия песен через запятую (или 'exit'): ").strip()
        if inp.lower() == "exit":
            break
        
        # Нормализация ввода: удаление пробелов и пустых значений
        names = [x.strip() for x in inp.split(',') if x.strip()]
        if not names:
            continue  # Пропуск пустого ввода
        
        # Генерация рекомендаций
        recs = recommender.recommend_by_names(names, n=10)
        
        if recs:
            print("\nТоп-10 рекомендаций:")
            # Форматированный вывод с ранжированием, названием, артистами и score
            for i, rec in enumerate(recs, 1):
                print(f"{i:2d}. {rec['name']} — {rec['artists']} (score: {rec['score']:.3f})")
        else:
            # Обработка случая отсутствия рекомендаций
            print("Нет рекомендаций.")

if __name__ == "__main__":
    main()  # Точка входа в приложение